In [2]:
!pip install pycaret
# !/opt/conda-environments/rapids-stable/bin/python -m pip install --upgrade pycaret

     |████████████████████████████████| 263 kB 5.0 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 167 kB 50.7 MB/s 
     |████████████████████████████████| 80 kB 8.4 MB/s 
     |████████████████████████████████| 1.7 MB 35.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 33.5 MB/s 
     |████████████████████████████████| 6.8 MB 39.3 MB/s 
     |████████████████████████████████| 271 kB 48.1 MB/s 
     |████████████████████████████████| 14.4 MB 54 kB/s 
     |████████████████████████████████| 105 kB 57.9 MB/s 
     |████████████████████████████████| 2.0 MB 29.6 MB/s 
     |████████████████████████████████| 248 kB 67.0 MB/s 
     |████████████████████████████████| 636 kB 48.6 MB/s 
     |████████████████████████████████| 1.1 MB 29.6 MB/s 
     |███████████████████

In [4]:
 from google.colab import drive
drive.mount('/content/gdrive')
 
directory_path = '/content/gdrive/My Drive/Colab Notebooks/컨퍼런스/'

Mounted at /content/gdrive


In [3]:
import warnings
warnings.filterwarnings('ignore')
 
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
 
from sklearn.preprocessing import LabelEncoder

from pycaret.classification import *
from sklearn.metrics import log_loss
import random
 
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
 
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
 
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# from sklearn import svm
# from sklearn.ensemble import RandomForestClassifier

In [5]:
test = pd.read_csv(directory_path+'test.csv')
train = pd.read_csv(directory_path+'train.csv')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [7]:
train.isnull().sum()

index               0
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64

In [8]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [10]:
train.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


# 전처리

**1. index, FLAG_MOBIL, occyp_type: 칼럼 제거**
- FLAG_MOBIL: 모든 값이 1
- occyp_type: 결측값이 너무 많음

In [11]:
train.drop(['index', 'FLAG_MOBIL', 'occyp_type'], axis=1, inplace=True)
test.drop(['index', 'FLAG_MOBIL', 'occyp_type'], axis=1, inplace=True)

**2. DAYS_BIRTH, DAYS_EMPLOYED, begin_month: 연도 단위로 변환**

In [12]:
# 날짜 변수 : 데이터 수집 당시 (0)부터 역으로 셈, 
# DAYS_BIRTH: -1은 데이터 수집일 하루 전에 태어났음을 의미
# DAYS_EMPLOYED : 양수 값은 고용되지 않은 상태를 의미
# begin_month: -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미

In [13]:
train[['DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']].describe()

,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
count,26457.000000,26457.000000,26457.000000
mean,-15958.053899,59068.750728,-26.123294
std,4201.589022,137475.427503,16.559550
min,-25152.000000,-15713.000000,-60.000000
25%,-19431.000000,-3153.000000,-39.000000
50%,-15547.000000,-1539.000000,-24.000000
75%,-12446.000000,-407.000000,-12.000000
max,-7705.000000,365243.000000,0.000000


In [14]:
day_col=['DAYS_BIRTH', 'DAYS_EMPLOYED']
for col in day_col:
    train[col] = train[col].map(lambda x: 0 if x > 0 else x)
    test[col] = test[col].map(lambda x: 0 if x > 0 else x)
    train[col]=np.abs(train[col])/365
    test[col]=np.abs(test[col])/365

train['begin_month']=abs(train['begin_month'])/12
test['begin_month']=abs(test['begin_month'])/12

In [15]:
train[['DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']].describe()

,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
count,26457.000000,26457.000000,26457.000000
mean,43.720696,6.023369,2.176941
std,11.511203,6.493536,1.379962
min,21.109589,0.000000,0.000000
25%,34.098630,1.115068,1.000000
50%,42.594521,4.216438,2.000000
75%,53.235616,8.638356,3.250000
max,68.909589,43.049315,5.000000


**3. child_num, family_size: 이상치 제거**

In [16]:
# child_num, family_size 이상치 존재 -> IQR 이상치 제거 방식
def get_outlier(df=None, column=None, weight=1.5):
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [17]:
for df in [train]:
    outlier_idx = get_outlier(df=df, column='child_num', weight=1.5)
    df.drop(outlier_idx, axis=0, inplace=True)
    outlier_idx = get_outlier(df=df, column='family_size', weight=1.5)
    df.drop(outlier_idx, axis=0, inplace=True)

# 파생변수

**1. hardship: 부담지수**

In [18]:
train['hardship']=((train['DAYS_BIRTH']-40)**3)*train['child_num']
test['hardship']=((test['DAYS_BIRTH']-40)**3)*test['child_num']

**2. communication: phone + work_phone + email**

In [19]:
train['communication']=0
test['communication']=0
for col in ['phone','work_phone','email']:
    train['communication']+=train[col]
    train=train.drop(col,axis=1)
for col in ['phone','work_phone','email']:
    test['communication']+=test[col]
    test=test.drop(col,axis=1)

**3. before_EMPLOYED: 고용되기 전까지의 연 수**

In [20]:
train['before_EMPLOYED'] = train['DAYS_BIRTH'] - train['DAYS_EMPLOYED']
test['before_EMPLOYED'] = test['DAYS_BIRTH'] - test['DAYS_EMPLOYED']

**4. ability: 연 소득/(살아온 해+근무 해)**

In [21]:
train['ability'] = train['income_total'] / (train['DAYS_BIRTH'] + train['DAYS_EMPLOYED'])
test['ability'] = test['income_total'] / (test['DAYS_BIRTH'] + test['DAYS_EMPLOYED'])

**5. income_mean: 소득/가족 내 성인 수**

In [22]:
train['income_mean'] = train['income_total'] / train['family_size']
test['income_mean'] = test['income_total'] / test['family_size']

**6. adult_num: 가족 내 성인 인원**

In [23]:
train['adult_num'] = train['family_size'] - train['child_num']
test['adult_num'] = test['family_size'] - test['child_num']

## 파생변수 확인

In [24]:
train[['hardship', 'communication', 'before_EMPLOYED', 'ability', 'income_mean', 'adult_num']].describe()

,hardship,communication,before_EMPLOYED,ability,income_mean,adult_num
count,26088.000000,26088.000000,26088.000000,26088.000000,26088.000000,26088.000000
mean,-101.569349,0.610549,37.791937,4050.491827,102004.094181,1.765984
std,941.268403,0.760991,13.365087,2471.210808,73239.270648,0.424114
min,-8374.289057,0.000000,17.936986,450.953091,11100.000000,-1.000000
25%,0.000000,0.000000,26.558904,2378.061980,56250.000000,2.000000
50%,0.000000,0.000000,34.936986,3532.426886,78750.000000,2.000000
75%,0.000000,1.000000,46.745205,5086.189100,126000.000000,2.000000
max,37282.777205,3.000000,68.909589,45549.084859,900000.000000,2.000000


**- adult_num 이상치 제거**

In [25]:
train = train[(train['adult_num'] > 0)]
train = train.reset_index(drop = True)

# Scailing, Encoding

**1. gender, car, reality: 이진 변수 수치화**

In [26]:
train['gender'] = train['gender'].replace(['F','M'],[0,1])
test['gender'] = test['gender'].replace(['F','M'],[0,1])
print('gender :')
print(train['gender'].value_counts())
print('--------------')

print('Having a car or not : ')
train['car'] = train['car'].replace(['N','Y'],[0,1])
test['car'] = test['car'].replace(['N','Y'],[0,1])
print(train['car'].value_counts())
print('--------------')

print('Having house reality or not: ')
train['reality'] = train['reality'].replace(['N','Y'],[0,1])
test['reality'] = test['reality'].replace(['N','Y'],[0,1])
print(train['reality'].value_counts())

gender :
0    17465
1     8617
Name: gender, dtype: int64
--------------
Having a car or not : 
0    16221
1     9861
Name: car, dtype: int64
--------------
Having house reality or not: 
1    17552
0     8530
Name: reality, dtype: int64


**2. income_type, family_type, house_type: LabelEncoder**

In [27]:
categorical_feats = ['income_type', 'family_type', 'house_type']
enc = LabelEncoder()
for i in categorical_feats:
    enc.fit(train[i])
    train[i] = enc.transform(train[i])
    test[i] = enc.transform(test[i])

In [28]:
# !pip install category_encoders

# from category_encoders.ordinal import OrdinalEncoder

# categorical_feats = ['income_type', 'edu_type', 'family_type', 'house_type']

# encoder = OrdinalEncoder(categorical_feats)
# train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
# test[categorical_feats] = encoder.transform(test[categorical_feats])

**3. edu_type**

In [29]:
def edu_transform(data):
    edu_ = ['Lower secondary','Secondary / secondary special',
            'Incomplete higher','Higher education','Academic degree']
    if data == edu_[0]:
        return 0
    elif data == edu_[1]:
        return 1
    elif data == edu_[2]:
        return 2
    elif data == edu_[3]:
        return 3
    else: 
        return 4

In [30]:
def edu_inverse(data):
    edu_ = ['Lower secondary','Secondary / secondary special',
            'Incomplete higher','Higher education','Academic degree']
    if data == 0:
        return edu_[0]
    elif data == 1:
        return edu_[1]
    elif data == 2:
        return edu_[2]
    elif data == 3:
        return edu_[3]
    else: 
        return edu_[4]

In [31]:
train['edu_type'] = train['edu_type'].apply(edu_transform)
test['edu_type'] = test['edu_type'].apply(edu_transform)

**4. income_total: 로그 변환**

In [32]:
for df in [train,test]:
    df['income_total'] = np.log1p(1+df['income_total'])

# 확인

In [33]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,credit,hardship,communication,before_EMPLOYED,ability,income_mean,adult_num
0,0,0,0,0,12.218505,0,3,1,2,38.079452,12.901370,2.0,0.500000,1.0,-0.000000,0,25.178082,3972.081900,101250.0,2.0
1,0,0,1,1,12.419174,0,1,0,1,31.178082,4.219178,3.0,0.416667,1.0,-686.576638,1,26.958904,6992.066563,82500.0,2.0
2,1,1,1,0,13.017007,4,3,1,1,52.293151,12.147945,2.0,1.833333,2.0,0.000000,1,40.145205,6983.121466,225000.0,2.0
3,0,0,1,0,12.218505,0,1,1,1,41.336986,5.731507,2.0,3.083333,0.0,0.000000,1,35.605479,4302.240978,101250.0,2.0
4,0,1,1,0,11.967193,2,3,1,1,41.197260,5.767123,2.0,2.166667,2.0,0.000000,0,35.430137,3353.605180,78750.0,2.0


In [34]:
train.columns

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type',
       'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'family_size', 'begin_month', 'credit', 'hardship', 'communication',
       'before_EMPLOYED', 'ability', 'income_mean', 'adult_num'],
      dtype='object')

In [35]:
numerical_feats = ['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'hardship', 'communication', 'before_EMPLOYED', 'ability', 'income_mean', 'adult_num', 'credit']
categorical_feats = ['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type']

In [36]:
train[numerical_feats].describe()

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,hardship,communication,before_EMPLOYED,ability,income_mean,adult_num,credit
count,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000,26082.000000
mean,0.387355,12.022601,43.806417,6.014046,2.153784,2.176594,-101.611569,0.610536,37.792371,4050.416711,101981.838394,1.766429,1.519362
std,0.646817,0.480359,11.546402,6.511273,0.832651,1.379902,941.352510,0.760944,13.366223,2471.296004,73210.909982,0.423110,0.702652
min,0.000000,10.203666,21.109589,0.000000,1.000000,0.000000,-8374.289057,0.000000,17.936986,450.953091,11100.000000,1.000000,0.000000
25%,0.000000,11.707686,34.120548,1.098630,2.000000,1.000000,0.000000,0.000000,26.556849,2377.660857,56250.000000,2.000000,1.000000
50%,0.000000,11.967193,42.793151,4.202740,2.000000,2.000000,0.000000,0.000000,34.938356,3532.426886,78750.000000,2.000000,2.000000
75%,1.000000,12.323865,53.333562,8.610959,3.000000,3.250000,0.000000,1.000000,46.745205,5086.189100,126000.000000,2.000000,2.000000
max,2.000000,14.269767,68.909589,43.049315,4.000000,5.000000,37282.777205,3.000000,68.909589,45549.084859,900000.000000,2.000000,2.000000


In [37]:
train[categorical_feats].astype('category').apply(pd.Series.value_counts)

,gender,car,reality,income_type,edu_type,family_type,house_type
0,17465.0,16221.0,8530.0,6123.0,255.0,2108.0,102
1,8617.0,9861.0,17552.0,4436.0,17717.0,17866.0,23315
2,NaN,NaN,NaN,2118.0,1001.0,1527.0,806
3,NaN,NaN,NaN,7.0,7086.0,3481.0,189
4,NaN,NaN,NaN,13398.0,23.0,1100.0,425
5,NaN,NaN,NaN,NaN,NaN,NaN,1245


# 모델 비교

In [38]:
clf = setup(train, target = 'credit', train_size = 0.85)

,Description,Value
0,session_id,6296
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26082, 20)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


In [39]:
#logloss 적용 및 predict_proba로 제출하기 위해 metric 추가
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7fc2d682b830>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [40]:
#svm, ridge는 predict_proba 미지원으로 제외
best4 = compare_models(fold = 5, sort = 'logloss', n_select = 4, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7012,0.7229,0.4370,0.6921,0.6306,0.2549,0.3313,0.7542,1.038
gbc,Gradient Boosting Classifier,0.6917,0.6651,0.4093,0.6618,0.6058,0.2095,0.3012,0.7926,12.162
lda,Linear Discriminant Analysis,0.6422,0.6088,0.3364,0.5327,0.5070,0.0091,0.0430,0.8629,0.196
lr,Logistic Regression,0.6413,0.5165,0.3333,0.4113,0.5011,0.0000,0.0000,0.8871,2.282
nb,Naive Bayes,0.6312,0.5614,0.3350,0.4729,0.5099,0.0053,0.0113,0.9504,0.076
rf,Random Forest Classifier,0.6973,0.7532,0.5467,0.6785,0.6842,0.3759,0.3799,0.9868,3.780
ada,Ada Boost Classifier,0.6892,0.6287,0.4031,0.6143,0.5991,0.1956,0.2942,1.0805,1.102
et,Extra Trees Classifier,0.6733,0.7095,0.5272,0.6546,0.6607,0.3279,0.3313,2.7140,3.090
knn,K Neighbors Classifier,0.6198,0.6625,0.4712,0.5992,0.6069,0.2220,0.2241,3.9910,0.678
dt,Decision Tree Classifier,0.6136,0.6464,0.4998,0.6233,0.6181,0.2697,0.2700,12.9178,0.326


In [41]:
blended = blend_models(estimator_list = best4, fold = 5, optimize = 'logloss')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6917,0.6934,0.4055,0.6186,0.6021,0.2021,0.3036,0.8027
1,0.6908,0.6981,0.4039,0.6206,0.6001,0.1973,0.3015,0.8017
2,0.6915,0.6997,0.4055,0.7431,0.6015,0.2003,0.3046,0.8027
3,0.6885,0.6925,0.4016,0.6412,0.5974,0.1921,0.2936,0.8057
4,0.6901,0.6886,0.4031,0.7419,0.5991,0.1950,0.2992,0.8055
Mean,0.6905,0.6945,0.4039,0.6731,0.6000,0.1973,0.3005,0.8037
SD,0.0011,0.0040,0.0015,0.0572,0.0017,0.0036,0.0039,0.0016


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6956,0.6959,0.4087,0.64,0.6073,0.208,0.3191,0.7986
